In [1]:
# Install helmet-assignment helper code
# !pip install input/helmet-assignment-helpers/helmet-assignment-main/ > /dev/null 2>&1
from helmet_assignment.score import NFLAssignmentScorer, check_submission
from helmet_assignment.features import add_track_features

import numpy as np
import pandas as pd
import glob
import os
import cv2
import itertools
from sklearn.metrics import accuracy_score
from tqdm.auto import tqdm
from multiprocessing import Pool
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
import random
random.seed(42)
from matplotlib.pyplot import figure

import sys
sys.path.append('input/easydict-master/easydict-master/')
# https://github.com/mikel-brostrom/Yolov5_DeepSort_Pytorch
sys.path.append('input/yolov5-deepsort-pytorch/Yolov5_DeepSort_Pytorch-master/Yolov5_DeepSort_Pytorch-master/deep_sort_pytorch/')
from deep_sort.deep_sort import DeepSort
from utils.parser import get_config
from helmet_assignment.video import video_with_predictions
from IPython.display import Video, display

def find_nearest(array, value): # находит значение ближайшего est_frame-а к value (фрэйму)
    value = int(value)
    array = np.asarray(array).astype(int)
    idx = (np.abs(array - value)).argmin() # из номеров est_frame вычитаем frame, берем модуль. Находим индекс минимального элемента в массиве
    return array[idx] # значение ближайшего est_frame-а

def modern_posit(image_pts, world_pts, focal_length, center):
    nb_points = np.shape(image_pts)[0]

    # centered & scaled pixel coordinates
    centered_image = np.divide(np.subtract(image_pts, center), focal_length)
    ui = centered_image[:, 0]
    vi = centered_image[:, 1]

    # homogeneous world coordinates
    homogeneous_world_pts = np.append(world_pts, np.ones((nb_points, 1)), 1)

    # pseudo inverse
    object_mat = np.linalg.pinv(homogeneous_world_pts)
    #print(object_mat)

    converged = 0
    count = 0
    t_x = 0.0
    t_y = 0.0
    t_z = 0.0
    r1 = 0.0
    r2 = 0.0
    r3 = 0.0
    while converged == 0:
        # POS part of the algorithm
        # rotation vectors
        r1_t = np.matmul(object_mat, ui)
        r2_t = np.matmul(object_mat, vi)
        # 1/t_z1 is norm of r1_t
        t_z1 = 1 / np.linalg.norm(r1_t[0:3])
        # 1/tz_2 is norm of r2_t
        t_z2 = 1 / np.linalg.norm(r2_t[0:3])
        
        # geometric average
        t_z = np.sqrt(t_z1 * t_z2)
        
        r1_n = np.multiply(r1_t, t_z)
        r2_n = np.multiply(r2_t, t_z)
        r1 = r1_n[0:3]
        r2 = r2_n[0:3]
        r3 = np.cross(r1, r2)
        r3_t = np.append(r3, t_z)
        t_x = r1_n[3]
        t_y = r2_n[3]

        # Now update the z/T z or epsilon
        # then ui, vi
        epsilon_i = np.matmul(homogeneous_world_pts, np.divide(r3_t, t_z))
        old_ui = ui
        old_vi = vi
        ui = np.multiply(epsilon_i, centered_image[:, 0])
        vi = np.multiply(epsilon_i, centered_image[:, 1])

        # check for convergence
        delta_ui = ui - old_ui
        delta_vi = vi - old_vi
        delta = np.square(focal_length) * (np.square(np.linalg.norm(delta_ui)) + np.square(np.linalg.norm(delta_vi)))

        converged = 1 if count > 0 and delta < 1 else 0
        count = count + 1

    trans = np.array([t_x, t_y, t_z], np.float64)
    rot = np.array([r1, r2, r3], np.float64)
    return rot, trans



In [2]:
out = pd.read_csv('tstr/out.csv')
tracking_gp = pd.read_csv('tstr/tracking_gp.csv')

In [4]:
tracking = tracking_gp

'''
IN: результаты дипсорта с самым частым лэйблом и его значением, а также трекинг

'''
# print('out\n',out)
# print('tracking\n',tracking)
PLOT = False

track_confidence = {}
for cluster in out['deepsort_cluster'].unique(): # Пробегаемся по кластерам дипсорта
    if not np.isnan(cluster): # Если кластер не None, то есть после пропускаемых дипсортом кадров
        label_count = out[out['deepsort_cluster'] == cluster]['label_count_deepsort'].unique()[0] # количество появлений топ кластера
        num_frames = len(out[out['deepsort_cluster'] == cluster]) # число кадров
        track_confidence[cluster] = label_count/num_frames # как часто кластер появляется на кадре. Словарь Кластер:Частота

out['track_confidence'] = out['deepsort_cluster'].map(track_confidence) # Добавляем столбец с conf трека
outs_posit = []
errors = []
for f in [5]:#(1, out['frame'].max()+1): # пробегаемся по кадрам                
    # результаты дипсорта сортируем по conf кластера из первого цикла и берем топ7 по conf
    out_posit = out[out['frame'] == f].sort_values(['track_confidence'], ascending=False).copy() 
    out_posit_confident = out_posit.head(7).copy() 
    
    # находим в треке примерный кадр и фиксируем его трек
    nearest_frame_tracking = find_nearest(tracking['est_frame'].values, f) # int номер кадра
    tracking_posit = tracking[tracking['est_frame']==nearest_frame_tracking].copy() # трек этого кадра

    # Если игрок входит в  Топ7 из кластеров дипсорта по conf 
    tracking_posit_confident = tracking_posit[tracking_posit['player'].isin(out_posit_confident['label_deepsort'].values)].copy()
    tracking_posit_confident['label_posit'] = tracking_posit_confident['player'].copy() # лейбл игрока
    
    # Находим дипсорт лэйблы у которых больше одного шлема и складываем вout_posit_confident_duplicates 
    out_posit_confident['label_posit'] = out_posit_confident['label_deepsort'].copy() # лейбл посита делаем из лейбла дипсорта
    out_posit_confident_duplicates = out_posit_confident['label_deepsort'].value_counts() # считаем количество значений лэйблов дипсорта
    out_posit_confident_duplicates = out_posit_confident_duplicates[out_posit_confident_duplicates > 1] # если больше 1 то складываем в датафрейм

    # Разбираемся с дубликатами 
    dupl_label = "" # заглушка для дублируемого лэйбла
    dupl_label_list = list(out_posit_confident_duplicates.to_dict().keys()) # делаем словарь (спиоск?) из 2+ встречающихся лэйблов дипсорта
    if len(dupl_label_list) > 0: # список длинный
        dupl_label = dupl_label_list[0] # оставляем первый лэйбл из списка

    # Для 7 уверенных кластеров (лейблов) проставляем координаты из трека
    out_posit_confident_merged = out_posit_confident.merge(tracking_posit_confident, on='label_posit', 
                                                           suffixes = ['','_t'])[['video_frame', 'label_posit', 'x', 'y', 'x_t', 'y_t']].copy()

    # По сути дубликат 7 уверенных шлемов дипсорта
    to_find_index = out_posit_confident.reset_index(drop=True).copy()

    # Делаем двухмерные координаты из координат центров боксов. Трехмерные из координат трека плюс дополнительная зануленная координата
    points_2d = np.array([out_posit_confident_merged['x'].values, 720-out_posit_confident_merged['y'].values]).transpose()
    points_3d = np.array([out_posit_confident_merged['x_t'].values, out_posit_confident_merged['y_t'].values]).transpose()
    points_3d = np.append(points_3d, np.zeros((points_3d.shape[0], 1)), axis=1)

    num_of_entries = len(points_2d) # Количество шлемов в датафрейме из 7 топ шлемов (наверное их всегда <=7?)
    # Находим список индексов где находится дубликат dupl_label 
    dupl_list = to_find_index.index[to_find_index['label_deepsort'] == dupl_label].tolist()
    
    best_error = float("inf")
    best_matrix = None
    best_projected_points_2d_homogen = None
    if (len(out_posit_confident['label_deepsort']) - len(out_posit_confident['label_deepsort'].unique()) == 1):
        print('+++++++++++++++++++++++++++++++')
        for idx in dupl_list:
            points_2d = np.array([out_posit_confident_merged['x'].values, 720-out_posit_confident_merged['y'].values]).transpose()
            points_3d = np.array([out_posit_confident_merged['x_t'].values, out_posit_confident_merged['y_t'].values]).transpose()
            points_3d = np.append(points_3d, np.zeros((points_3d.shape[0], 1)), axis=1)
            good_indices = list(range(num_of_entries))  # [0, 1, 2, 3, 4, 6]
            good_indices.pop(idx)
            points_2d = np.array([points_2d[i] for i in good_indices])
            points_3d = np.array([points_3d[i] for i in good_indices])
            #rot, trans = modern_posit(points_2d[indexes], points_3d[indexes], 1920, [640, 360])
            rot, trans = modern_posit(points_2d, points_3d, 1920, [640, 360])
            rottrans = np.append(rot, trans.reshape(3,1), axis=1)
            intrin = np.array([
                [1920, 0, 640],
                [0, 1920, 360],
                [0, 0, 1],
            ])
            full_matrix = np.matmul(intrin, rottrans)
            projected_points_2d = np.matmul(full_matrix, np.transpose(np.append(points_3d, np.ones((points_3d.shape[0], 1)), axis=1)))
            projected_points_2d_t = np.transpose(projected_points_2d)
            projected_points_2d_homogen = projected_points_2d_t / projected_points_2d[2].reshape(points_3d.shape[0],1)
            norm_error = np.linalg.norm(points_2d - projected_points_2d_homogen[:,0:2])
            if norm_error < best_error:
                best_error = norm_error
                best_matrix = full_matrix
                best_projected_points_2d_homogen = projected_points_2d_homogen
        norm_error = best_error
        full_matrix = best_matrix
        projected_points_2d_homogen = best_projected_points_2d_homogen
    else:
        rot, trans = modern_posit(points_2d, points_3d, 1920, [640, 360])
        rottrans = np.append(rot, trans.reshape(3,1), axis=1)
        intrin = np.array([
            [1920, 0, 640],
            [0, 1920, 360],
            [0, 0, 1],
        ])
        full_matrix = np.matmul(intrin, rottrans)
        projected_points_2d = np.matmul(full_matrix, np.transpose(np.append(points_3d, np.ones((points_3d.shape[0], 1)), axis=1)))
        projected_points_2d_t = np.transpose(projected_points_2d)
        projected_points_2d_homogen = projected_points_2d_t / projected_points_2d[2].reshape(points_3d.shape[0],1)
        norm_error = np.linalg.norm(points_2d - projected_points_2d_homogen[:,0:2])

    if norm_error == float("inf"):
        norm_error = 0.0
    #print(f, 'norm:', norm_error, f,nearest_frame_tracking, len(out_posit_confident), len(out_posit_confident['label_deepsort']) - len(out_posit_confident['label_deepsort'].unique()))
    errors.append(norm_error)

    if PLOT:
        plt.plot(points_2d[:, 0], points_2d[:, 1], 'rx')
        plt.plot(projected_points_2d_homogen[:, 0], projected_points_2d_homogen[:, 1], 'b.')
        plt.axis('scaled')
        plt.show()

        x = projected_points_2d_homogen[:, 0]
        y = projected_points_2d_homogen[:, 1]
        labels = out_posit_confident_merged['label_posit'].values
        labels_posit = out_posit_confident['label_posit'].values
        fig, ax = plt.subplots()
        fig.set_size_inches((15, 15))
        ax.scatter(points_2d[:, 0], points_2d[:, 1], marker='x', color='r')
        ax.scatter(x, y, marker='.', color='b')
        for i in range(len(x)):
            ax.annotate(labels[i], (x[i], y[i]), color='b')

        for i in range(points_2d.shape[0]):
            ax.annotate(labels_posit[i], (points_2d[:, 0][i], points_2d[:, 1][i]), color='r')


    points_2d_full = np.array([out_posit['x'].values, 720-out_posit['y'].values]).transpose()
    points_3d_full = np.array([tracking_posit['x'].values, tracking_posit['y'].values]).transpose()
    points_3d_full = np.append(points_3d_full, np.zeros((points_3d_full.shape[0], 1)), axis=1)

    projected_points_2d_full = np.matmul(full_matrix, np.transpose(np.append(points_3d_full, np.ones((points_3d_full.shape[0], 1)), axis=1)))
    projected_points_2d_t_full = np.transpose(projected_points_2d_full)
    projected_points_2d_homogen_full = projected_points_2d_t_full / projected_points_2d_full[2].reshape(points_3d_full.shape[0],1)
    tracking_posit["x_pr"] = projected_points_2d_homogen_full[:, 0]
    tracking_posit["y_pr"] = projected_points_2d_homogen_full[:, 1]

    if PLOT:
        figure(figsize=(10,10))
        plt.plot(points_2d_full[:, 0], points_2d_full[:, 1], 'rx')
        plt.plot(projected_points_2d_homogen_full[:, 0], projected_points_2d_homogen_full[:, 1], 'b.')
        plt.axis('scaled')
        plt.show()

        x = projected_points_2d_homogen_full[:, 0]
        y = projected_points_2d_homogen_full[:, 1]
        labels = tracking_posit['player'].values
        labels_posit = out_posit['label'].values
        fig, ax = plt.subplots()
        ax.axis('equal')

        fig.set_size_inches((30, 30))
        ax.scatter(points_2d_full[:, 0], points_2d_full[:, 1], marker='x', color='r')
        ax.scatter(x, y, marker='.', color='b')
        for i in range(len(x)):
            ax.annotate(labels[i], (x[i], y[i]), color='b')

        for i in range(len(labels_posit)):
            ax.annotate(labels_posit[i], (points_2d_full[:, 0][i], points_2d_full[:, 1][i]), color='r')


    out_posit['label_posit'] = out_posit['label_deepsort'].values.copy()
    out_posit['y_im'] = (720 - out_posit['y']).copy()
    indexes_to_all_distances ={}
    for index, row in out_posit.iterrows():
        all_distances = np.sqrt(np.square(tracking_posit['x_pr'].values - row['x']) + np.square(tracking_posit['y_pr'].values - row['y_im']))
        all_labels = tracking_posit['player'].values 
        indexes_to_all_distances[index] = sorted(list(zip(all_labels, all_distances)), key=lambda tup: tup[1])


    all_results = {}
    expected_num_of_results = len(indexes_to_all_distances)
    for _ in range(expected_num_of_results):
        min_element = None
        min_dist = float('inf')
        min_label = None
        for k, v in indexes_to_all_distances.items():
            if v[0][1] < min_dist:
                min_dist = v[0][1]
                min_element = k
                min_label = v[0][0]
        if min_element:
            all_results[min_element] = min_label
        indexes_to_all_distances.pop(min_element, None)
        for k, v in indexes_to_all_distances.items():
            indexes_to_all_distances[k] = [t for t in v if t[0] != min_label]

    for k, v in all_results.items():
        out_posit.loc[k,'label_posit'] = v

    outs_posit.append(out_posit.copy())

submission_posit = pd.concat(outs_posit).copy()



+++++++++++++++++++++++++++++++
